In [ ]:
import threading
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
import tensorflow as tf
#sess = tf.InteractiveSession()
tf.set_random_seed(123)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
queue_input_data  = tf.placeholder(tf.float32, shape=[100, 784])
queue_input_label = tf.placeholder(tf.float32, shape=[100, 10])

queue = tf.FIFOQueue(capacity=400, dtypes=[tf.float32, tf.float32], shapes=[[784], [10]])

enqueue_op = queue.enqueue_many([queue_input_data, queue_input_label])
dequeue_op = queue.dequeue()

# tensorflow recommendation:
# capacity = min_after_dequeue + (num_threads + a small safety margin) * batch_size
data_batch, label_batch = tf.train.batch(dequeue_op, batch_size=50, capacity=400)

def enqueue(sess):
    while True:
        batch = mnist.train.next_batch(100)
        sess.run(enqueue_op, feed_dict={queue_input_data : batch[0],
                                        queue_input_label : batch[1]})
        #print("added to the queue")
    print("finished enqueueing\n")


In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
images  = data_batch
labels  = label_batch
image2d = tf.reshape(images, [-1, 28, 28, 1])

In [ ]:

# convolution layer
with tf.variable_scope("conv1"):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(image2d, W_conv1) + b_conv1)
with tf.variable_scope("pool1"):
    h_pool1 = max_pool_2x2(h_conv1)
# convolution layer  
with tf.variable_scope("conv2"):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
with tf.variable_scope("pool2"):
    h_pool2 = max_pool_2x2(h_conv2)
# fc layer
with tf.variable_scope("fc1"):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# fc layer 
with tf.variable_scope("fc2"):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    prediction = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
# train operation
with tf.variable_scope("train"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=prediction))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.variable_scope("analysis"):
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    softmax = tf.nn.softmax(logits=prediction)
    prediction_label = tf.argmax(prediction,1)


In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(tf.global_variables_initializer())

enqueue_thread = threading.Thread(target=enqueue, args=[sess])
enqueue_thread.daemon = True
enqueue_thread.start()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)
run_options = tf.RunOptions(timeout_in_ms=4000) 

In [ ]:
# Training w/ 5000 steps
import time
start = time.time()
for i in range(5000):
    if i % 100 == 0:
        train_accuracy, = sess.run([accuracy],feed_dict={keep_prob: 1.0},options=run_options)
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run([train_step],feed_dict={keep_prob: 0.5},options=run_options)

acc, = sess.run([accuracy],feed_dict={keep_prob:1.0}, options=run_options)    
print('test accuracy %g' % acc)
print(time.time()-start)     

In [ ]:
# Dump some output images
image2d, prob_array, pred_label, = sess.run([image2d,softmax,prediction_label],feed_dict={keep_prob:1.0},options=run_options)
for index in xrange(10):
    print 'Prediction\033[91m',pred_label[index],'\033[00mwith softmax prob\033[94m',prob_array[index][pred_label[index]],'\033[00m'
    plt.imshow(image2d[index].reshape([28,28]).astype(np.float32),interpolation='none')
    plt.axis('off')
    plt.show()

